### En primer lugar se creará el conector a SQL

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import os  

engine = create_engine(os.path.join('sqlite:///','data', 'database_challenge.db'))

In [2]:
query ="""
SELECT name 
FROM sqlite_master 
WHERE type='table'
"""
df = pd.read_sql(query, con=engine)
df

,name
0,upl_students
1,upl_courses
2,upl_campuses
3,upl_academicperiods
4,upl_enrollment_statuses
5,reg_enrollments


### Definimos la función de consultas

In [3]:
def consulta(query,conn):
    return pd.read_sql_query(query, con=conn)

In [4]:
conn = create_engine(os.path.join('sqlite:///','data','database_challenge.db'))

##### (a) Generamos la siguiente tabla para ver la cantidad de alumnos matriculados por cada periodo académico. 

In [5]:
query = """
SELECT 
COUNT(DISTINCT id_student) AS cantidad_alumnos, id_academicPeriod as periodo_academico, is_active AS activo
FROM reg_enrollments
WHERE is_active='1'
GROUP BY id_academicPeriod
"""
consulta(query,conn)
#No consideramos los periodos con is_active=0 ya que son registros que no están en uso

,cantidad_alumnos,periodo_academico,activo
0,1026,1,1
1,1013,3,1
2,637,5,1
3,461,6,1
4,798,8,1
5,1323,12,1
6,1537,13,1
7,1080,15,1
8,1727,18,1
9,1132,20,1


##### Se genera una segunda tabla para conocer cuales son los periodos académicos correspondientes

In [6]:
query = """
SELECT id_code AS codigo,ds_integrationId AS periodo_academico,is_active AS activo FROM upl_academicperiods
WHERE is_active='1'
"""
consulta(query,conn)

,codigo,periodo_academico,activo
0,1,20151S,1
1,3,20162S,1
2,5,20141S,1
3,6,20142S,1
4,8,20152S,1
5,12,20161S,1
6,13,20171S,1
7,15,20172S,1
8,18,20181S,1
9,20,20182S,1


##### Con ambas tablas notamos que por cada año tenemos registros activos solo en periodos semestrales, además, la cantidad de estudiantes matriculados por cada semestre de cada año es:
* 2014 1S: 637 - 2014 2S: 461
* 2015 1S: 1026 - 2015 2S: 798
* 2016 1S: 1323 - 2016 2S: 1013 
* 2017 1S: 1537 - 2017 2S: 1080
* 2018 1S: 1727 - 2018 2S: 1132

##### por lo que podemos ver que en cada año, el primer semestre hay más estudiantes matriculados que el segundo semestre, y que cada año se van matriculando más estudiantes

##### (b) generamos una tabla que nos muestre cuantos ramos tomó cada alumno en cada periodo de tiempo

In [7]:
query2="""
SELECT 
id_academicPeriod AS periodo_academico, COUNT(id_course) AS cantidad_cursos FROM reg_enrollments
WHERE is_active='1'
GROUP BY id_academicPeriod , id_student
"""
consulta(query2,conn)

,periodo_academico,cantidad_cursos
0,1,4
1,1,4
2,1,4
3,1,4
4,1,4
...,...,...
10729,20,4
10730,20,5
10731,20,5
10732,20,4


##### Luego, por cada periodo de tiempo obtenemos el promedio de ramos inscritos

In [8]:
per=consulta(query2,conn)['periodo_academico'].unique()
prom=0
for p in per:
    df_aux = consulta(query2,conn).loc[lambda x: x['periodo_academico'] == p].cantidad_cursos.mean()
    print('En el periodo académico',p, 'el promedio de ramos inscritos es:', df_aux)

En el periodo académico 1 el promedio de ramos inscritos es: 3.9717348927875245
En el periodo académico 3 el promedio de ramos inscritos es: 4.815399802566634
En el periodo académico 5 el promedio de ramos inscritos es: 3.0
En el periodo académico 6 el promedio de ramos inscritos es: 3.0
En el periodo académico 8 el promedio de ramos inscritos es: 4.0
En el periodo académico 12 el promedio de ramos inscritos es: 4.799697656840514
En el periodo académico 13 el promedio de ramos inscritos es: 4.69811320754717
En el periodo académico 15 el promedio de ramos inscritos es: 4.734259259259259
En el periodo académico 18 el promedio de ramos inscritos es: 4.699478865083961
En el periodo académico 20 el promedio de ramos inscritos es: 4.7871024734982335


##### Utilizando la misma tabla de los periodos de tiempo, tenemos que el promedio de ramos por cada semestre (aproximados a la unidad) serían:
* 2014 1S: 3 - 2014 2S: 3
* 2015 1S: 4 - 2015 2S: 4
* 2016 1S: 5 - 2016 2S: 5 
* 2017 1S: 5 - 2017 2S: 5
* 2018 1S: 5 - 2018 2S: 5

##### Y vemos que la cantidad de ramos promedio inscritos por los alumnos fue aumentando en el periodo de 2014-2016 para luego mantenerse en 5 ramos inscritos en promedio.

##### (c) Generamos la tabla para contar cuantos reprobados hay por cada curso y la ordenamos de mayor a menor

In [9]:
query3="""
SELECT id_course AS curso, COUNT(id_status) AS cantidad_reprobados FROM reg_enrollments
WHERE (id_academicPeriod='1') AND (id_status=2) AND (is_active='1')
GROUP BY id_course
ORDER BY COUNT(id_status) DESC

"""
consulta(query3,conn)

,curso,cantidad_reprobados
0,3421,67
1,3416,53
2,3437,20
3,2357,13
4,1250,12
...,...,...
141,1018,1
142,1001,1
143,997,1
144,984,1


##### Así obetenemos los 5 ramos con mayor tasa de reprobación, luego generamos una segunda tabla para revisar cuales son los cursos mencionados

In [10]:
query4="""
SELECT id_code, ds_name FROM upl_courses
WHERE (id_code='3421') OR (id_code='3416') OR (id_code='3437') OR (id_code='2357') OR (id_code='1250')
"""
consulta(query4,conn)

,id_code,ds_name
0,1250,Inglés 1
1,2357,Inglés 1
2,3416,Introducción A La Física Universitaria
3,3421,Introducción A La Matemática Universitaria
4,3437,Química General Básica


##### Por lo que los 5 ramos con mayor tasa de reprobación en el periodo 2015 1S son:
* Introducción A La Matemática Universitaria (3421): 67 reprobados
* Introducción A La Física Universitaria (3416): 53 reprobados
* Química General Básica (3437): 20 reprobados
* Inglés 1 (2357): 13 reprobados
* Inglés 1 (1250): 12 reprobados